In [1]:
import os
import random
from solve import Formula

In [2]:
def create_random_config(number_of_variables):
    config = []
    for _ in range(number_of_variables):
        if random.random() < 0.5:
            config.append(1)
        else:
            config.append(0)
    return config

In [3]:
filename = os.path.join(os.getcwd(), "data", "wuf20-71-M", "wuf20-01.mwcnf")
formula = Formula(filename)
variable_configuration = create_random_config(formula.number_of_variables)
print(formula)
print("Assignment: ", variable_configuration)

Clauses: [[4, -18, 19], [3, 18, -5], [-5, -8, -15], [-20, 7, -16], [10, -13, -7], [-12, -9, 17], [17, 19, 5], [-16, 9, 15], [11, -5, -14], [-18, 14, 1], [-19, -15, 10], [-8, 4, 7], [7, 17, -15], [12, -7, -14], [2, -15, -11], [9, 6, 1], [9, -15, 13], [12, -7, -17], [-18, -2, 20], [20, 12, 4], [-16, 18, -4], [-13, 15, 10], [12, -14, -7], [-7, 16, 10], [6, 10, 7], [20, 14, -16], [-19, 17, 11], [-7, 1, -20], [-5, 12, 15], [12, -11, -7], [-5, 19, -8], [1, 16, 17], [20, -14, -15], [13, -4, 10], [14, 7, 10], [-5, 9, 20], [10, 1, -19], [-16, -15, -1], [-15, -10, 4], [4, -15, -3], [-10, -16, 11], [-8, 12, -5], [14, -6, 12], [1, 6, 11], [-13, -5, -1], [-7, -2, 12], [1, -20, 19], [-2, -13, -8], [-11, 14, 9], [-6, -15, -2], [5, -12, -15], [-6, 17, 5], [-5, 19, -18], [-18, 14, -4], [9, -5, -1], [10, -19, -14], [20, 9, 4], [-5, 13, -17], [-18, 3, 11], [7, -9, 17], [-15, -6, -3], [-2, 3, -13], [12, 3, -2], [-2, -3, 17], [20, -15, -16], [-5, -17, -19], [-20, -18, 11], [-9, 1, -5], [-19, 9, 17], [12, -